<h1><center>Classification II: tutorial</center></h1>

<h3>Importing packages</h3>

You should have `numpy`, `pandas`, `matplotlib` and `seaborn` installed from the previous session.

Today we'll use additional functions from `sklearn` and `xgboost`.

E.g. install the latter as (or your preferred method):
> conda install -c anaconda py-xgboost

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as mx

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # for Lasso and Elastic Net logistic regression
from sklearn.svm import SVC # for Support Vector Classification
from sklearn.feature_selection import RFE # for feature selection in SVM
from sklearn.ensemble import RandomForestClassifier # for Random Forests
from sklearn.ensemble import GradientBoostingClassifier # for Gradient Boosted Trees
from sklearn.manifold import MDS # for visualising proximity matrix
from xgboost import XGBClassifier # for XGBoost


<h3>Setting parameters</h3>

To make it different for everyone, you'll use different parameters for the different algorithms.

For the random seed write your CID without leading zeroes, e.g. instead of 01234567 you write 1234567 below.

Choose your test set size, number of random iterations/trees (depending on model) and your n top most important variables to print as well.
Test set size is a fraction between 0 and 1, the training data will be the remainder.

In [ ]:
randomSeed = 709878 # add your CID here without leading zeroes

testSize = 1/5 # feel free to change this anywhere between 0.05 and 0.5

nIter = 42 # feel free to change this, but don't go overboard - still need to do the computation in a reasonable time...

nImpVars = 20 # feel free to change this

<h3>Your assignments during the tutorial</h3>

In [ ]:
np.random.RandomState(seed=randomSeed)
classIImethodsA = ['Lasso', 'Random Forest', 'SVM-RFE']
classIImethodsA = np.random.choice(classIImethodsA, size=3, replace=False)
print('First do',classIImethodsA[0])
print('Then do',classIImethodsA[1])
print('Followed by',classIImethodsA[2])
print('\n')
classIImethodsB = ['Elastic Net', 'Gradient Boosted Decision Trees and XGBoost', 'Nonlinear SVM-RFE']
classIImethodsB = np.random.choice(classIImethodsB, size=3, replace=False)
print('Then, if there is time left, do',classIImethodsB[0])
print('Followed by',classIImethodsB[1])
print('Last do',classIImethodsB[2])

<h3>Importing data</h3>

In [ ]:
QMDP_data = pd.read_excel('QMDiab_metabolomics_Preprocessed.xlsx', sheet_name='plasma')

In [ ]:
print(QMDP_data)

In [ ]:
X = QMDP_data.iloc[0:, 1:323].to_numpy() # identified metabolites
y = QMDP_data.iloc[0:, 506].to_numpy() # type-2 diabetes status
Xlabs = np.array(list(QMDP_data.columns[1:323]))
p = Xlabs.size

<h3>Function and class to save and calculate performance using confusion matrix as input</h3>

We'll use this for all algorithms to have a standardised output.

In [ ]:
def modelPerformance(confMat):
    TN = confMat[0, 0]
    TP = confMat[1, 1]
    FP = confMat[0, 1]
    FN = confMat[1, 0]
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    spec = TN / (TN + FP)
    fpr = FP / (TN + FP)
    f1 = 2 * (prec * rec) / (prec + rec)
    acc = (TP + TN) / (TP + FP + TN + FN)
    return (acc, prec, rec, spec, fpr, f1)
def printPerformance(confMat):
    acc, prec, rec, spec, fpr, f1 = modelPerformance(confMat)
    print("Accuracy = " "%.4f" % acc)
    print("Precision = " "%.4f" % prec)
    print("Recall = " "%.4f" % rec)
    print("Specificity = " "%.4f" % spec)
    print("False positive rate = " "%.4f" % fpr)
    print("F1-score = " "%.4f" % f1)
    np.set_printoptions(precision=2)
    print("Confusion matrix (%):")
    print(confMat/np.sum(confMat)*100)

In [ ]:
class Result:
    def __init__(self, cmat, impVar):
        self.cmat = cmat
        self.acc, self.prec, self.rec, self.spec, self.fpr, self.f1 = modelPerformance(cmat)
        self.impVar = impVar

<h3>Choose your model and setup</h3>

You will find mocks for running a single model below for each of the examples methods.

E.g. for penalised logistic regression, SVM and XGBoost this is one model, for random forests this is one ensemble of `nIter` trees. If you want to run multiple permutations of a model, have a look at the example for ridge logistic regression and implement it for penalised logistic regression, SVM and/or XGBoost.

<b> Have a look at the comments behind functions, some of these indicate you <i>may</i> want to optimise/change some (hyper)parameters. Use the <a href="https://scikit-learn.org/stable/supervised_learning.html#supervised-learning">online documentation</a> in</b> `sklearn` <b>for classification to see how to do this.</b> Specifically, look at [`GridSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [`RandomizedSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

[`GridSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) goes through all possible combinations of parameters, this can be very slow if you aim to evaluate a large set of possible parameters values.

The [`RandomizedSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) function evaluates the same set of parameters but instead you input the number of iterations (```n_iter```) which indicates how many random (remember to set the ```random_state```!) combinations are being evaluated. The higher the ```n_iter``` the more combinations will be evaluated, but also the slower it gets. Use this to balance the computation time. Set ```n_iter``` as your input value of _nIter_.

It is fine to use the default values in this session if you have not used these methods before, if you have used them before then implement the hyperparameter tuning with one of these two methods.

<h3>Ridge logistic regression with multiple permutations of the data</h3>

Could also use `LogisticRegressionCV` (instead of `LogisticRegression`) here, see the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html">documentation online</a>.

In [ ]:
cmat = np.zeros((2, 2), dtype=int)
Bmat = np.zeros((p,nIter))
for k in range(0, nIter):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=(randomSeed+k))
    mdl = LogisticRegression(penalty='l2',random_state=(randomSeed-k),solver='liblinear').fit(X_train, y_train) # not using the lbfgs solver due to a bug
    y_test_hat = mdl.predict(X_test)
    cmat += mx.confusion_matrix(y_test, y_test_hat)
    Bmat[:,k] = mdl.coef_[0]

In [ ]:
printPerformance(cmat)

In [ ]:
# get important variables (e.g. order based on average weights)
Baw = np.mean(Bmat, axis=1)
idx = (-abs(Baw)).argsort()[:nImpVars]
print(Xlabs[idx])

In [ ]:
# save results to use compare with later
result_RR = Result(cmat,Xlabs[idx])

<h3>Lasso logit regression</h3>

Adapt this code to include multiple permutations of the model (or use one that already includes cross-validation, see the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html">documentation online</a>).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
mdl = LogisticRegression(penalty='l1',random_state=(randomSeed-1),solver='liblinear').fit(X_train, y_train)
y_test_hat = mdl.predict(X_test)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
# get important variables (non-zero coefficients)
B = mdl.coef_[0]
idx = (-abs(B)).argsort()[:nImpVars]
print(Xlabs[idx])

In [ ]:
# save results to use compare with later
result_lasso = Result(cmat,Xlabs[idx])

<h3>Random Forest</h3>

Adapt this code to change the parameters used to calculate the forest.

E.g. is there a difference in performance of using gini impurity compared to using entropy?

In [ ]:
def proximityMatrix(mdl, X, normalize=True):      
    endNode = mdl.apply(X)
    nTrees = endNode.shape[1]
    n = X.shape[0]
    proxMat = np.zeros((n,n))
    for k in range(0, nTrees):
        a = endNode[:,k]
        proxMat += 1*np.equal.outer(a, a)
    if normalize:
        proxMat = proxMat / nTrees
    return proxMat

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
mdl = RandomForestClassifier(n_estimators=nIter, criterion='gini', random_state=randomSeed)
mdl = mdl.fit(X_train, y_train)
y_test_hat = mdl.predict(X_test)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
ind = (y_test).argsort()
xM = X_test[ind,]
yM = y_test[ind]
proxMat = proximityMatrix(mdl, xM, normalize=True)
ax = plt.axes()
ax = sns.heatmap(proxMat, vmin=0, vmax=1, ax = ax)
ax.set_title('Heatmap of ordered proximity matrix')
plt.show()

In [ ]:
emb = MDS(n_components=2,dissimilarity='precomputed',metric=True)
S = emb.fit_transform(1-proxMat)
S = pd.DataFrame({"MDS_1":S[:,0], "MDS_2":S[:,1]})
sns.scatterplot(data = S, x = 'MDS_1', y = 'MDS_2', hue = yM)
plt.title('MDS visualisation of proximity matrix')
plt.show()

In [ ]:
# get important variables
idx = (-(mdl.feature_importances_)).argsort()[:nImpVars]
print(Xlabs[idx])

In [ ]:
# save results to use compare with later
result_RF = Result(cmat,Xlabs[idx])

<h3>Linear Support Vector Classification</h3>

Adapt this code to include multiple permutations of the model (or use one that already includes cross-validation).

Next you can try to use another classifier (such as `NuSVM`), or to see <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV">documentation online</a> for finding the optimal number of features.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
mdl = SVC(kernel='linear',C=1) # is not optimising the cost...
mdl.fit(X_train, y_train)
y_test_hat = mdl.predict(X_test)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
# get important variables (beta weights)
B = mdl.coef_[0]
idx = (-abs(B)).argsort()[:nImpVars]
print(Xlabs[idx])

In [ ]:
# save results to use compare with later
result_linearSVM = Result(cmat,Xlabs[idx])

In [ ]:
# perform RFE on the data to find 10 most important variables
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
est = SVC(kernel='linear',C=1) # is not optimising the cost...
mdl = RFE(est, n_features_to_select=nImpVars, step=.1) # does not do cross-validation to select the optimal number of features
mdl.fit(X_train, y_train)
# get important variables (from variable selection using RFE)
idx = mdl.ranking_==1
print(Xlabs[idx])
mdl = SVC(kernel='linear',C=1) # is not optimising the cost...
xtr = X_train[:,idx]
xte = X_test[:,idx]
mdl.fit(xtr, y_train)
y_test_hat = mdl.predict(xte)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
# save results to use compare with later
result_linearSVMRFE = Result(cmat,Xlabs[idx])

<h3>Support Vector Classification with non-linear kernel</h3>

Choose a non-linear kernel (see documentation online).

Adapt this code to include multiple permutations of the model (or use one that already includes cross-validation).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
mdl = SVC(kernel='rbf',C=1,gamma='scale') # is not optimising the cost...
mdl.fit(X_train, y_train)
y_test_hat = mdl.predict(X_test)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
acc, prec, rec, spec, fpr, f1 = modelPerformance(cmat) # calculate model values

In [ ]:
# get important variables (remove 1 variable at a time and evaluate drop in performance as measure or importance)
deltaAcc = np.zeros((1,p))
for k in range(0,p):
    mdlMinK = SVC(kernel='rbf',C=1,gamma='scale') # is not optimising the cost...
    xtr = np.delete(X_train,k,1)
    xte = np.delete(X_test,k,1)
    mdlMinK.fit(xtr, y_train)
    y_test_hat = mdlMinK.predict(xte)
    cmatMinK = mx.confusion_matrix(y_test, y_test_hat)
    accMinK, precMinK, recMinK, specMinK, fprMinK, f1MinK = modelPerformance(cmatMinK) # calculate model values
    deltaAcc[0,k] = acc - accMinK

In [ ]:
idx = (-deltaAcc[0]).argsort()[:nImpVars]
print(Xlabs[idx])

In [ ]:
# save results to use compare with later
result_nonlinearSVM = Result(cmat,Xlabs[idx])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
mdl = SVC(kernel='rbf',C=1,gamma='scale') # is not optimising the cost, or other hyperparameters...
# we're taking the top 10 most important variables based on accuracy drop, but perhaps as with linear-RFE there is another optimum?
# sklearn's RFE only works for linear kernels, hence the workaround above (but you should be to see how one could write it themselves)
xtr = X_train[:,idx]
xte = X_test[:,idx]
mdl.fit(xtr, y_train)
y_test_hat = mdl.predict(xte)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
# save results to use compare with later
# it's technically not RFE what was done, since variable accuracy drop was defined relative to all other variables
# it would have to be repeated multiple times in steps as with linear RFE (however would take a lot of time)
# but after this session you could try and code it?
result_nonlinearSVMRFE = Result(cmat,Xlabs[idx])

<h3>Gradient Boosted Decision Trees and XGBoost</h3>

Set/optimize the hyperparameters (see documentation online for <a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html">GBDT</a> and <a href="https://xgboost.readthedocs.io/en/latest/parameter.html">XGB</a>).

Adapt this code to include multiple permutations of the model (or use one that already includes cross-validation).

In [ ]:
# Gradient Boosted Trees
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
mdl = GradientBoostingClassifier(n_estimators=nIter, learning_rate=1.0, max_depth=1, random_state=(randomSeed-1))
mdl.fit(X_train, y_train)
y_test_hat = mdl.predict(X_test)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
# get important variables
idx = (-mdl.feature_importances_).argsort()[:nImpVars]
print(Xlabs[idx])

In [ ]:
# save results to use compare with later
result_GBDT = Result(cmat,Xlabs[idx])

In [ ]:
# XGBoost
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
mdl = XGBClassifier(subsample=2/3,objective='reg:logistic',seed=(randomSeed-1))
mdl.fit(X_train, y_train)
y_test_hat = mdl.predict(X_test)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
# get important variables
idx = (-mdl.feature_importances_).argsort()[:nImpVars]
print(Xlabs[idx])

In [ ]:
# save results to use compare with later
result_XGBoost = Result(cmat,Xlabs[idx])

<h3>Elastic Net logit regression</h3>

Optimize the hyperparameters. Below it is only done for a single alpha value (`l1_ratio`), this is rarely optimal.
E.g. use one that already includes cross-validation (see the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html">documentation online</a>).

Adapt this code to include multiple permutations of the model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomSeed)
mdl = LogisticRegression(penalty='elasticnet',random_state=(randomSeed-1),solver='saga',l1_ratio=0.5,max_iter=5000).fit(X_train, y_train)
y_test_hat = mdl.predict(X_test)
cmat = mx.confusion_matrix(y_test, y_test_hat)

In [ ]:
printPerformance(cmat)

In [ ]:
# get important variables (non-zero coefficients)
B = mdl.coef_[0]
idx = (-abs(B)).argsort()[:nImpVars]
print(Xlabs[idx])

In [ ]:
# save results to use compare with later
result_EN = Result(cmat,Xlabs[idx])

<h3>Recap</h3>

This tutorial was intended to give you <i>some</i> experience with these models and to compare the outcomes.

However it does not mean that your best model here is always the best model to use for other data.

With all of these models, there are many different parameters to choose from to change the outcome of the model.

Have a look in the documentation for some of these and see if you can improve on your best model? Or try <a href="https://scikit-learn.org/stable/supervised_learning.html#supervised-learning">other supervised models</a>.

Best practice is to split the data in 3 ways: training, validation and text. Training is to build your models, validation is to pick the optimal hyperparameters, and test is used to give an unbiased estimate of the performance of the model with chosen hyperparameters.

In [ ]:
print('F1-scores:')
print("Ridge: " "%.4f" % result_RR.f1)
print("Lasso: " "%.4f" % result_lasso.f1)
print("Elastic Net: " "%.4f" % result_EN.f1)
print("linear SVM: " "%.4f" % result_linearSVM.f1)
print("linear SVM-RFE: " "%.4f" % result_linearSVMRFE.f1)
print("Non-linear SVM: " "%.4f" % result_nonlinearSVM.f1)
print("Non-linear SVM-RFE: " "%.4f" % result_nonlinearSVMRFE.f1)
print("Random Forest: " "%.4f" % result_RF.f1)
print("GBDT: " "%.4f" % result_GBDT.f1)
print("XGBoost: " "%.4f" % result_XGBoost.f1)

<h3>Do after this session: stratified analysis</h3>

Do the analyses you did above for women and men separately, do the results change (predictive ability, important variables)?

You can also look at associations of <u>individual</u> (important) variables to see if they are different based on sex e.g. using differences in regression coefficients as you did in <i>Introduction to Statistical Learning in Python</i>.